# 17 QListWidget

The **QListWidget** class allows you to create a list view widget that has a single column of items. The **QListWidgetItem** class represents the items on the list.

The QListWidget class has various useful methods for manipulating items including:

- addItems(iterable)  adds items to the list from an iterable of strings.
- addItem(QListWidgetItem)  adds an item to the end of the list.
- insertItem(row, QListWidgetItem)  inserts an item at the specified row.
- takeItem(row)  removes an item from a specified row.
- clear()  removes and deletes all items from the list.

## 17.1 First example

In [1]:
import sys
from PyQt6.QtWidgets import QInputDialog, QApplication, QWidget,  QGridLayout, QListWidget,  QPushButton
from PyQt6.QtGui import QIcon


class MainWindow(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.setWindowTitle('My Wish List')
        self.setGeometry(100, 100, 400, 100)

        layout = QGridLayout(self)
        self.setLayout(layout)

        self.list_widget = QListWidget(self)
        self.list_widget.addItems(['Learn Python', 'Master PyQt'])
        layout.addWidget(self.list_widget, 0, 0, 4, 1)

        # create buttons
        add_button = QPushButton('Add')
        add_button.clicked.connect(self.add)

        insert_button = QPushButton('Insert')
        insert_button.clicked.connect(self.insert)

        remove_button = QPushButton('Remove')
        remove_button.clicked.connect(self.remove)

        clear_button = QPushButton('Clear')
        clear_button.clicked.connect(self.clear)

        layout.addWidget(add_button, 0, 1)
        layout.addWidget(insert_button, 1, 1)
        layout.addWidget(remove_button, 2, 1)
        layout.addWidget(clear_button, 3, 1)

        # show the window
        self.show()

    def add(self):
        text, ok = QInputDialog.getText(self, 'Add a New Wish', 'New Wish:')
        if ok and text:
            # add item to the end of the list
            self.list_widget.addItem(text)

    def insert(self):
        text, ok = QInputDialog.getText(self, 'Insert a New Wish', 'New Wish:')
        if ok and text:
            # get current row
            current_row = self.list_widget.currentRow()
            # insert new row just after the current row
            self.list_widget.insertItem(current_row+1, text)

    def remove(self):
        current_row = self.list_widget.currentRow()
        if current_row >= 0:
            # get current row item
            current_item = self.list_widget.takeItem(current_row)
            # delete the current row item
            del current_item

    def clear(self):
        # delete all items
        self.list_widget.clear()


app = QApplication(sys.argv)
window = MainWindow()
sys.exit(app.exec())

qt.qpa.xcb: X server does not support XInput 2


SystemExit: 0

/home/pengfei/.cache/pypoetry/virtualenvs/python-desktop-app-framework-xY3vli_t-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 17.2 Use model in QTreeWidget

In above example, we add item to list manually. As We have shown in `03.QTableView`, we can use a model to pass data into a QTableView. We can do the same by using QListView. For QListView, the model needs to extend the `QAbstractListModel`.

Below is an example which use model and QListView. The functionalities are the same as above example

In [ ]:
import sys
from PyQt6.QtCore import QAbstractListModel, Qt
from PyQt6.QtWidgets import QInputDialog, QApplication, QWidget, QGridLayout, QListWidget, QPushButton, QListView


class ListModel(QAbstractListModel):
    def __init__(self, itemList, *args, **kwargs):
        super(ListModel, self).__init__(*args, **kwargs)
        if itemList:
            self.itemList = itemList
        else:
            self.itemList = []

    def data(self, index, role):
        if role == Qt.ItemDataRole.DisplayRole:
            status, text = self.itemList[index.row()]
            return text

    def rowCount(self, index):
        return len(self.itemList)


class MainWindow(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.setWindowTitle('My Wish List')
        self.setGeometry(100, 100, 400, 100)

        layout = QGridLayout(self)
        self.setLayout(layout)

        # note only QListView can use model by calling .setModel
        self.listView = QListView(self)
        # a list of tuple, first param is the status of the wish, second is the text of the wish
        self.itemList = [(False, 'Learn Python'), (False, 'Master PyQt')]
        self.model = ListModel(self.itemList)
        self.listView.setModel(self.model)
        layout.addWidget(self.listView, 0, 0, 4, 1)

        # create buttons
        add_button = QPushButton('Add')
        add_button.clicked.connect(self.add)

        insert_button = QPushButton('Insert')
        insert_button.clicked.connect(self.insert)

        remove_button = QPushButton('Remove')
        remove_button.clicked.connect(self.remove)

        clear_button = QPushButton('Clear')
        clear_button.clicked.connect(self.clear)

        layout.addWidget(add_button, 0, 1)
        layout.addWidget(insert_button, 1, 1)
        layout.addWidget(remove_button, 2, 1)
        layout.addWidget(clear_button, 3, 1)

        # show the window
        self.show()

    def add(self):
        # as we use QListView, we can't use addItem anymore
        # here we modify directly the model
        text, ok = QInputDialog.getText(self, 'Add a New Wish', 'New Wish:')
        if ok and text:
            # add item to the model
            # by default, the status of the wish is False
            self.model.itemList.append((False, text))
            # we need to trigger the refresh event
            self.model.layoutChanged.emit()

    def insert(self):
        text, ok = QInputDialog.getText(self, 'Insert a New Wish', 'New Wish:')
        if ok and text:
            # get current row
            current_row = self.listView.currentIndex().row()
            # insert new row just after the current row
            self.model.itemList.insert(current_row + 1, (False, text))
            # we need to trigger the refresh event
            self.model.layoutChanged.emit()

    def remove(self):
        current_row = self.listView.currentIndex().row()
        if current_row >= 0:
            # delete current row item
            del self.model.itemList[current_row]
            # we need to trigger the refresh event
            self.model.layoutChanged.emit()

    def clear(self):
        # delete all items
        self.model.itemList.clear()
        # we need to trigger the refresh event
        self.model.layoutChanged.emit()


app = QApplication(sys.argv)
window = MainWindow()
sys.exit(app.exec())


## 17.3 Using Qt.ItemDataRole.DecorationRole

In above example, you have noticed that each wish has a `status` boolean variable. We will use it to mark wish if it comes true. We will add a button `mark` to change the status.

In below example, the first change is in `ListModel`. We add some code to treat the `DecorationRole`

```python
if role == Qt.ItemDataRole.DecorationRole:
            status, _ = self.itemList[index.row()]
            if status:
                return tick
```

Second change is the `mark` slot which modifies the model, then notify the view, the model has been changed.

You can find the complete code example below.

In [ ]:
import sys

from PyQt6 import QtGui
from PyQt6.QtCore import QAbstractListModel, Qt
from PyQt6.QtWidgets import QInputDialog, QApplication, QWidget, QGridLayout, QPushButton, QListView

tick = QtGui.QImage('../../resources/tick.png')


class ListModel(QAbstractListModel):
    def __init__(self, itemList, *args, **kwargs):
        super(ListModel, self).__init__(*args, **kwargs)
        if itemList:
            self.itemList = itemList
        else:
            self.itemList = []

    def data(self, index, role):
        if role == Qt.ItemDataRole.DisplayRole:
            status, text = self.itemList[index.row()]
            return text
        if role == Qt.ItemDataRole.DecorationRole:
            status, _ = self.itemList[index.row()]
            if status:
                return tick

    def rowCount(self, index):
        return len(self.itemList)


class MainWindow(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.setWindowTitle('My Wish List')
        self.setGeometry(100, 100, 400, 100)

        layout = QGridLayout(self)
        self.setLayout(layout)

        # note only QListView can use model by calling .setModel
        self.listView = QListView(self)
        # a list of tuple, first param is the status of the wish, second is the text of the wish
        self.itemList = [(True, 'Learn Python'), (False, 'Master PyQt')]
        self.model = ListModel(self.itemList)
        self.listView.setModel(self.model)
        layout.addWidget(self.listView, 0, 0, 4, 1)

        # create buttons
        add_button = QPushButton('Add')
        add_button.clicked.connect(self.add)

        insert_button = QPushButton('Insert')
        insert_button.clicked.connect(self.insert)

        remove_button = QPushButton('Remove')
        remove_button.clicked.connect(self.remove)

        mark_button = QPushButton('Mark')
        mark_button.clicked.connect(self.mark)

        clear_button = QPushButton('Clear')
        clear_button.clicked.connect(self.clear)

        layout.addWidget(add_button, 0, 1)
        layout.addWidget(insert_button, 1, 1)
        layout.addWidget(remove_button, 2, 1)
        layout.addWidget(mark_button, 3, 1)
        layout.addWidget(clear_button, 4, 1)

        # show the window
        self.show()

    def add(self):
        # as we use QListView, we can't use addItem anymore
        # here we modify directly the model
        text, ok = QInputDialog.getText(self, 'Add a New Wish', 'New Wish:')
        if ok and text:
            # add item to the model
            # by default, the status of the wish is False
            self.model.itemList.append((False, text))
            # we need to trigger the refresh event
            self.model.layoutChanged.emit()

    def insert(self):
        text, ok = QInputDialog.getText(self, 'Insert a New Wish', 'New Wish:')
        if ok and text:
            # get current row
            current_row = self.listView.currentIndex().row()
            # insert new row just after the current row
            self.model.itemList.insert(current_row + 1, (False, text))
            # we need to trigger the refresh event
            self.model.layoutChanged.emit()

    def remove(self):
        current_row = self.listView.currentIndex().row()
        if current_row >= 0:
            # delete current row item
            del self.model.itemList[current_row]
            # we need to trigger the refresh event
            self.model.layoutChanged.emit()

    def mark(self):
        indexes = self.listView.selectedIndexes()
        if indexes:
            index = indexes[0]
            rowNb = index.row()
            # get the selected wish value
            status, text = self.model.itemList[rowNb]
            # overwrite the wish with status as True
            self.model.itemList[rowNb] = (True, text)
            # .dataChanged takes top-left and bottom right, which are equal
            # for a single selection.
            self.model.dataChanged.emit(index, index)
            # Clear the selection (as it is no longer valid).
            self.listView.clearSelection()

    def clear(self):
        # delete all items
        self.model.itemList.clear()
        # we need to trigger the refresh event
        self.model.layoutChanged.emit()


app = QApplication(sys.argv)
window = MainWindow()
sys.exit(app.exec())
